In [1]:
import QuantLib as ql
import pandas as pd

## Swap Rates from Bloomberg

In [2]:
today = ql.Date(17, 8, 2023)
ql.Settings.instance().evaluationDate = today

data = pd.DataFrame({
    "Term": ["1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M", "12M", "18M", "2Y", "3Y", "4Y"],
    "Rate": [5.30111, 5.30424, 5.30657, 5.31100, 5.34800, 5.38025, 5.40915, 5.43078, 5.44235, 5.44950, 5.44878, 5.44100, 5.42730, 5.40747, 5.3839, 5.09195, 4.85785, 4.51845, 4.31705],
})

## Fitting the SOFR Curve

In [3]:
# Params
calendar = ql.UnitedStates(ql.UnitedStates.SOFR)
frequency = ql.Annual
convention = ql.ModifiedFollowing
day_count = ql.Actual360()
settlement = 2
settle_date = calendar.adjust(today + settlement)

In [4]:
# Termination dates for swap rates
data['Termination'] = [calendar.adjust(settle_date + ql.Period(t), ql.Following) for t in data['Term']]
data

,Term,Rate,Termination
0,1W,5.30111,"August 28th, 2023"
1,2W,5.30424,"September 5th, 2023"
2,3W,5.30657,"September 11th, 2023"
3,1M,5.31100,"September 21st, 2023"
4,2M,5.34800,"October 23rd, 2023"
5,3M,5.38025,"November 21st, 2023"
6,4M,5.40915,"December 21st, 2023"
7,5M,5.43078,"January 22nd, 2024"
8,6M,5.44235,"February 21st, 2024"
9,7M,5.44950,"March 21st, 2024"


In [5]:
# SOFR Index Object
sofr_index = ql.OvernightIndex(
    "SOFR", 
    0, 
    ql.USDCurrency(), 
    calendar, 
    day_count
)

# SOFR swap helpers for bootstrapping the curve
helpers = []

for _, row in data.iterrows():
    helper = ql.OISRateHelper(
        settlement,
        ql.Period(row['Termination'] - settle_date, ql.Days),
        ql.QuoteHandle(ql.SimpleQuote(row['Rate']/100)),
        sofr_index,
        paymentLag=2,
        paymentConvention=convention,
        paymentFrequency=frequency,
        paymentCalendar=calendar,
    )
    helpers.append(helper)

In [6]:
# Fit a piece-wise log-linear discount curve using the helpers
curve = ql.PiecewiseLogLinearDiscount(today, helpers, day_count)

In [7]:
# Display the discount factors
pd.DataFrame({
    'Date': data['Termination'],
    'Discount Factor': [curve.discount(d) for d in data['Termination']],
})

,Date,Discount Factor
0,"August 28th, 2023",0.998382
1,"September 5th, 2023",0.997208
2,"September 11th, 2023",0.996327
3,"September 21st, 2023",0.994862
4,"October 23rd, 2023",0.990145
5,"November 21st, 2023",0.985856
6,"December 21st, 2023",0.981421
7,"January 22nd, 2024",0.976721
8,"February 21st, 2024",0.972364
9,"March 21st, 2024",0.968194


## Pricing a SOFR OIS Swap

In [8]:
# Parameters
effective = ql.Date(21, 11, 2023)
termination = ql.Date(21, 2, 2025)
notional = 100e6
swap_type = ql.Swap.Receiver
fixed_rate = 0.054
spread = 0
payment_lag = 2

In [9]:
# Accrual schedule for swap
fixed_schedule = ql.Schedule(
    effective, 
    termination,
    ql.Period(frequency),
    calendar,
    convention,
    convention,
    ql.DateGeneration.Backward,
    False
)

# Create sofr index object with the fitted sofr curve
sofr_index = ql.OvernightIndex(
    "SOFR", 0, ql.USDCurrency(), calendar, day_count, ql.YieldTermStructureHandle(curve)
)

# Create swap
def create_swap(rate, engine):
    swap = ql.OvernightIndexedSwap(
        swap_type,
        notional,
        fixed_schedule,
        rate,
        day_count,
        sofr_index,
        spread,
        payment_lag,
        convention
    )
    swap.setPricingEngine(engine)
    return swap

engine = ql.DiscountingSwapEngine(ql.YieldTermStructureHandle(curve))
swap = create_swap(fixed_rate, engine)

In [10]:
# Par coupon and NPV
par_coup = round(swap.fairRate() * 100, 6)
npv = round(swap.NPV(), 2)

In [11]:
# PV01
swap_up = create_swap(fixed_rate + 0.0001, engine)
swap_down = create_swap(fixed_rate - 0.0001, engine)
npv_up = swap_up.NPV()
npv_down = swap_down.NPV()
pv01 = round((npv_up - npv_down) / 2, 2)

In [12]:
# DV01 & Gamma
# Function for constructing new curve with spread
def npv_with_bump(bump):
    # New index
    new_index = ql.OvernightIndex("SOFR", 0, ql.USDCurrency(), calendar, day_count)
    
    # New helpers
    new_helpers = []
    for _, row in data.iterrows():
        helper = ql.OISRateHelper(
            settlement,
            ql.Period(row['Termination'] - settle_date, ql.Days),
            ql.QuoteHandle(ql.SimpleQuote(row['Rate']/100+bump)),
            new_index,
            paymentLag=2,
            paymentConvention=convention,
            paymentFrequency=frequency,
            paymentCalendar=calendar,
        )
        new_helpers.append(helper)
        
    # New curve
    new_curve = ql.PiecewiseLogLinearDiscount(today, new_helpers, day_count)
    
    # Create sofr index object with the fitted sofr curve
    new_index = ql.OvernightIndex(
        "SOFR", 0, ql.USDCurrency(), calendar, day_count, ql.YieldTermStructureHandle(new_curve)
    )
    
    # Create swap
    new_swap = ql.OvernightIndexedSwap(
        swap_type,
        notional,
        fixed_schedule,
        fixed_rate,
        day_count,
        new_index,
        spread,
        payment_lag,
        convention
    )
    new_engine = ql.DiscountingSwapEngine(ql.YieldTermStructureHandle(new_curve))
    new_swap.setPricingEngine(new_engine)
    return round(new_swap.NPV(), 2)

npv_up = npv_with_bump(0.0001)
npv_down = npv_with_bump(-0.0001)
dv01 = round((npv_down - npv_up) / 2, 2)

normalized_gamma = round((npv_up - 2 * npv + npv_down) / (0.0001**2) / notional, 2)

In [13]:
# Display results
df = pd.DataFrame(
    [par_coup, npv, pv01, dv01, normalized_gamma],
    index=["Par Coupon (%)", "NPV ($)", "PV01", "DV01", "Normalized Gamma"],
    columns=["Value"]
)
df

,Value
Par Coupon (%),5.016152
NPV ($),456605.720000
PV01,11895.480000
DV01,11879.420000
Normalized Gamma,3.190000


In [14]:
# Cashflows
leg_0 = swap.leg(0)
leg_1 = swap.leg(1)

def extract_cashflows(leg):
    flows = {}
    for cf in leg:
        if cf.hasOccurred():
            continue  # skip past cashflows
        date = cf.date()
        amount = cf.amount()
        flows[date] = flows.get(date, 0.0) + amount
    return flows

leg_0_flows = extract_cashflows(leg_0)
leg_1_flows = extract_cashflows(leg_1)

# Merge into DataFrame
all_dates = sorted(set(leg_0_flows.keys()) | set(leg_1_flows.keys()))
row_data = []
for d in all_dates:
    leg_0_amt = round(leg_0_flows.get(d, 0.0), 2)
    leg_1_amt = -round(leg_1_flows.get(d, 0.0), 2)
    net_amt = leg_0_amt + leg_1_amt
    discount = curve.discount(d)
    pv = round(net_amt * discount, 2)
    row_data.append([ql.Date.to_date(d), leg_0_amt, leg_1_amt, net_amt, discount, pv])

df = pd.DataFrame(row_data, columns=["Pay Date", "Receive", "Pay", "Net", "Discount", "PV"])
df

,Pay Date,Receive,Pay,Net,Discount,PV
0,2024-02-23,1380000.0,-1387613.74,-7613.74,0.972074,-7401.12
1,2025-02-25,5490000.0,-4988750.77,501249.23,0.925701,464006.84
